# Sample Notebook - AIDC with OpenScale


### Import AIDC library and requirements

In [2]:
## Import AIDC libraries
!pip install /project_data/data_asset/aidc-2.0.zip;
!pip install --upgrade ibm-watson-machine-learning   | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
import aidc

Processing /project_data/data_asset/aidc-2.0.zip
  Preparing metadata (setup.py) ... done
  Using cached javascript-1%211.0.3-py3-none-any.whl (33 kB)
  Created wheel for aidc: filename=aidc-2.0-py3-none-any.whl size=138109 sha256=d37c095c3affe5086ce7c85819dc67fa730b466c6e0665eb6134c3b0e14f0217
  Stored in directory: /tmp/1000710000/.cache/pip/wheels/36/35/e2/ffafef2711229494b6f5514e8d265d330c9b11bfa3285f4fce
Successfully built aidc
  Attempting uninstall: javascript
    Found existing installation: javascript 1!1.0.3
    Uninstalling javascript-1!1.0.3:
      Successfully uninstalled javascript-1!1.0.3
  Attempting uninstall: aidc
    Found existing installation: aidc 2.0
    Uninstalling aidc-2.0:
      Successfully uninstalled aidc-2.0


### Configure WML client to deploy the function

In [3]:
#Configure the WML client
import os
from ibm_watson_machine_learning import APIClient

WOS_CREDENTIALS = {
    "url": os.environ.get("RUNTIME_ENV_APSX_URL"),
    "username": "admin",
    "apikey": "wTCDCBGZkMvCv9wfMqNHX38HIz65zuKxDZsZFJiM",
}

#Subscription ID of the model from OpenScale
subscription_id = "40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8"

#Default guid
WOS_GUID="00000000-0000-0000-0000-000000000000"

WML_CREDENTIALS = WOS_CREDENTIALS.copy()
WML_CREDENTIALS['instance_id']='openshift'
WML_CREDENTIALS['version']='4.7'

PYTHON_FUNCTION_NAME = 'AIDC Custom Metric Function'
DEPLOYMENT_NAME = 'AIDC Custom Metric Deployment'
CUSTOM_METRICS_PROVIDER_NAME = "AIDC Metrics Provider"
CUSTOM_MONITOR_NAME = 'Aidc Demo monitor'

### Connect to the space

In [4]:
#Configure the space
wml_client = APIClient(WML_CREDENTIALS)

metadata = {            
     wml_client.spaces.ConfigurationMetaNames.NAME: 'AIDC_space',         
     wml_client.spaces.ConfigurationMetaNames.DESCRIPTION:  'AIDC space',            
     }
SPACE_NAME=metadata[wml_client.spaces.ConfigurationMetaNames.NAME]

def guid_from_space_name(client, space_name):
 space = client.spaces.get_details()
 try:
     return_item=next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id']
 except:
     return_item="0"
 return(return_item)

space_uid = guid_from_space_name(wml_client, SPACE_NAME)
if(space_uid=="0"):
    space_details = wml_client.spaces.store(meta_props=metadata,background_mode=False)
    space_uid = guid_from_space_name(wml_client, SPACE_NAME)
wml_client.set.default_space(space_uid)

'SUCCESS'

### Define the monitor logic

In [5]:
#Custom metric functions
parms = {
        "url": WOS_CREDENTIALS["url"],
        "username": WOS_CREDENTIALS["username"],
        "apikey": WOS_CREDENTIALS["apikey"]
    }
def custom_metrics_provider(parms = parms):
    
    ######################## Edit the values for your custom performance model. ############################
    TruePositiveCost = "0"
    FalsePositiveCost = "-LoanAmount*0.03"
    FalseNegativeCost = "-LoanAmount*0.5"
    TrueNegativeCost = "LoanAmount*0.3"
    ModelDecisionCost = "-0.02"
    HumanDecisionCost = "-0.05"
    #################################################################################################
        
    import json
    import requests
    import base64
    from requests.auth import HTTPBasicAuth
    import time
    import uuid
    import datetime
    import aidc
    import pandas as pd
    
    headers = {}
    headers["Content-Type"] = "application/json"
    headers["Accept"] = "application/json"
    
    
    # Get the access token
    def get_access_token():
        url = '{}/icp4d-api/v1/authorize'.format(parms['url'])
        payload = {
            'username': parms['username'],
            'api_key': parms['apikey']
        }
        response = requests.post(url, headers=headers, json=payload, verify=False)
        json_data = response.json()
        access_token = json_data['token']
        return access_token 
    
    #Update the run status to Finished in the Monitor Run
    def update_monitor_run_status(base_url, access_token, custom_monitor_instance_id, run_id, status, error_msg = None):
        monitor_run_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/runs/'+run_id
        completed_timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")
        patch_payload  = []
        base_path = "/status"        
        patch_payload.append(get_patch_request_field(base_path, "state", status))
        patch_payload.append(get_patch_request_field(base_path, "completed_at", completed_timestamp))
        if error_msg != None:
            error_json = get_error_json(error_msg)
            patch_payload.append(get_patch_request_field(base_path, "failure", error_json))        
        headers["Authorization"] = "Bearer {}".format(access_token)
        response = requests.patch(monitor_run_url, headers=headers, json = patch_payload, verify=False)
        monitor_run_response = response.json()
        return response.status_code, monitor_run_response
    
    
    def get_error_json(error_message):
        trace = str(uuid.uuid4())
        error_json = {
            'trace': trace,
            'errors': [{
                'code': "custom_metrics_error_code",
                'message': str(error_message)
            }]
        }
        return error_json
    
    def get_patch_request_field(base_path, field_name, field_value, op_name="replace"):
        field_json = {
            "op": op_name,
            "path": "{0}/{1}".format(base_path, field_name),
            "value": field_value
        }
        return field_json
    
    def sumPerformance(payload_data,taskmodel):
        total_value=0
        probability_position=payload_data['records'][0]['fields'].index("probability")        

        for i in range(len(payload_data['records'][0]['values'])):
            probability=payload_data["records"][0]["values"][i][probability_position][0]
            difference=aidc.sumPerProbability(taskmodel,probability)
            total_value+=difference 
    
        return total_value
    
    def collect_feedback_dataset(access_token, data_mart_id, feedback_dataset_id):        
        offset = 0
        limit = 1000
        reading_data = True
        json_data = None
        annotations = {"annotations": []}
        fields = {"fields": []}
        values = {"values": []}        
        result = None        
        while reading_data:
            if feedback_dataset_id is not None:
                headers["Authorization"] = "Bearer {}".format(access_token)
                DATASETS_STORE_RECORDS_URL = parms["url"] + "/openscale/{0}/v2/data_sets/{1}/records?offset={2}&limit={3}&format=list".format(data_mart_id, feedback_dataset_id, offset, 1000)
                response = requests.get(DATASETS_STORE_RECORDS_URL, headers=headers, verify=False)
                json_data = response.json()                
                offset += 1000                
                if len(json_data["records"]) != 0:                                
                    annotations["annotations"] = json_data["records"][0]["annotations"]                    
                    fields["fields"] = json_data["records"][0]["fields"]                                
                    for val in json_data["records"][0]["values"]:                        
                        values["values"].append(val)                                                                    
                if len(json_data["records"]) == 0: 
                    reading_data = False                                        
        result = {"records":[{"annotations": annotations["annotations"], "fields": fields["fields"], "values": values["values"]}]}        
        return result
    
    def collect_payload_dataset(access_token, data_mart_id, payload_dataset_id):
        json_data = None
        if payload_dataset_id is not None:
            headers["Authorization"] = "Bearer {}".format(access_token)
            DATASETS_STORE_RECORDS_URL = parms["url"] + "/openscale/{0}/v2/data_sets/{1}/records?limit={2}&format=list".format(data_mart_id, payload_dataset_id, 100)
            response = requests.get(DATASETS_STORE_RECORDS_URL, headers=headers, verify=False)
            json_data = response.json()
        return json_data
    
    def get_metrics(access_token, data_mart_id, subscription_id, feedback_dataset_id,payload_dataset_id):        
        json_data = collect_feedback_dataset(access_token, data_mart_id, feedback_dataset_id)        
        payload_data = collect_payload_dataset(access_token, data_mart_id, payload_dataset_id)        
        
        number_of_requests=0
        decisioncost_value=0
        performance_value=0
        impact_value=0
        ml_value=0
        human_value=0
        roi=0
        if json_data is not None and len(json_data["records"][0]['values'])>0:
            fields = json_data['records'][0]['fields']
            values = json_data['records'][0]['values']
            feedback_data = pd.DataFrame(values, columns = fields)
            table = aidc.load_pandas_data(feedback_data)        
            taskmodel_data={
              'id': "0",
              'name': "taskModel",
              'description': "aidc"
            }
            taskmodel=aidc.create_task_model(table,taskmodel_data)                        
            aidc.set_custom_indicators(taskmodel, TruePositiveCost, FalsePositiveCost, FalseNegativeCost, TrueNegativeCost, ModelDecisionCost, HumanDecisionCost)                        
            metrics = aidc.get_indicators(taskmodel)              
            performance_value = float(metrics["Performance"])            
            impact_value = float(metrics["Impact"])            
            decisioncost_value = float(metrics["Decision Cost"])            
            ml_value = float(metrics["ml volume"])            
            human_value = float(metrics["human volume"])            
        
        if payload_data is not None and len(payload_data["records"])>0 and performance_value>0:
           roi=sumPerformance(payload_data,taskmodel)
            
        metrics = {"decisioncost": decisioncost_value,
                   "performance": performance_value, 
                   "impact": impact_value, 
                   "ml": ml_value, 
                   "human": human_value,
                   "roi": roi}
        return metrics
          
    # Publishes the Custom Metrics to OpenScale
    def publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitoring_run_id, feedback_dataset_id, timestamp,payload_dataset_id):
        # Generate an monitoring run id, where the publishing happens against this run id
        custom_metrics = get_metrics(access_token, data_mart_id, subscription_id, feedback_dataset_id,payload_dataset_id)
        measurements_payload = [
                  {
                    "timestamp": timestamp,
                    "run_id": custom_monitoring_run_id,
                    "metrics": [custom_metrics]
                  }
                ]
        headers["Authorization"] = "Bearer {}".format(access_token)
        measurements_url = base_url + '/v2/monitor_instances/' + custom_monitor_instance_id + '/measurements'
        response = requests.post(measurements_url, headers=headers, json = measurements_payload, verify=False)
        published_measurement = response.json()
        return response.status_code, published_measurement
        
    
    def publish( input_data ):        
        timestamp = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")        
        payload = input_data.get("input_data")[0].get("values")
        data_mart_id = payload['data_mart_id']
        subscription_id = payload['subscription_id']
        custom_monitor_id = payload['custom_monitor_id']
        custom_monitor_instance_id = payload['custom_monitor_instance_id']
        custom_monitor_instance_params  = payload['custom_monitor_instance_params']
        custom_monitor_run_id = payload['custom_monitor_run_id']
        payload_dataset_id = payload.get('payload_dataset_id')
        feedback_dataset_id = payload.get('feedback_dataset_id')
        base_url = parms['url'] + '/openscale' + '/' + data_mart_id
        access_token = get_access_token()        
        published_measurements = []
        error_msgs = []
        run_status = "finished"        
        try:
            last_run_time = custom_monitor_instance_params.get("last_run_time")
            max_records = custom_monitor_instance_params.get("max_records")
            min_records = custom_monitor_instance_params.get("min_records")
            error_msg = None            
            status_code, published_measurement = publish_metrics(base_url, access_token, data_mart_id, subscription_id, custom_monitor_id, custom_monitor_instance_id, custom_monitor_run_id, feedback_dataset_id, timestamp, payload_dataset_id)
            if int(status_code) in [200, 201, 202]:
                published_measurements.append(published_measurement)
            else:
                run_status = "error"
                error_msg = published_measurement
                error_msgs.append(error_msg)                    
            status_code, response = update_monitor_run_status(base_url, access_token, custom_monitor_instance_id, custom_monitor_run_id, run_status, error_msg)            
            if not int(status_code) in [200, 201, 202]:
                error_msgs.append(response)                
        except Exception as ex:
            error_msgs.append(str(ex))
        if len(error_msgs) == 0:
            response_payload = {
                "predictions" : [{ 
                    "values" : published_measurements
                }]
            }
        else:
            response_payload = {
                "predictions":[],
                "errors": error_msgs
            }        
        return response_payload
        
    return publish

### Prepare software specifications

In [6]:
#Prepare and deploy the custom function

#Clean previous deployments
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
        
#Find the AIDC custom software definition
file_path_AIDC = '/project_data/data_asset/aidc-2.0.zip'
specs_list = wml_client.software_specifications.get_details()
for spec in specs_list["resources"]:
    name=(spec['metadata']['name'])
    if("aidc-spec" in name):
        print(name)
        base_software_spec_id =  wml_client.software_specifications.get_id_by_name(name)


# Package extension metadata
meta_prop_pkg_ext = {
     wml_client.package_extensions.ConfigurationMetaNames.NAME: "AIDC_custom_package",
     wml_client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "AIDC custom package",
     wml_client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
}
package_ext_details = wml_client.package_extensions.store(meta_props=meta_prop_pkg_ext, file_path = file_path_AIDC)
package_ext_uid = wml_client.package_extensions.get_uid(package_ext_details)

# Prepare software specificaion
meta_prop_sw_spec = {
     wml_client.software_specifications.ConfigurationMetaNames.NAME: "AIDC_software_spec",
     wml_client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "AIDC software specification",
     wml_client.software_specifications.ConfigurationMetaNames.PACKAGE_EXTENSIONS : [{
         "guid": package_ext_uid
     }],
     wml_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
         "guid": base_software_spec_id
     }
}
software_spec_details = wml_client.software_specifications.store(meta_props=meta_prop_sw_spec)
software_spec_uid = wml_client.software_specifications.get_uid(software_spec_details)

#Function meta details
meta_props = {
         wml_client.repository.FunctionMetaNames.NAME: PYTHON_FUNCTION_NAME,
         wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_ID: software_spec_uid
}

#Prepare the function
function_artifact = wml_client.repository.store_function(meta_props=meta_props, function=custom_metrics_provider)
function_uid = wml_client.repository.get_function_id(function_artifact)

#Prepare hardware specs
hardware_spec_id = wml_client.hardware_specifications.get_id_by_name('S')

#Prepare the deployment metadata
deploy_meta = {
     wml_client.deployments.ConfigurationMetaNames.NAME: DEPLOYMENT_NAME,
     wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
     wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { "id": hardware_spec_id}
}

#Deploy the function
deployment_details = wml_client.deployments.create(function_uid, meta_props=deploy_meta)

created_at = deployment_details['metadata']['created_at']
find_string_pos = created_at.find("T")
if find_string_pos != -1:
    current_date = created_at[0:find_string_pos]
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
scoring_url = scoring_url + "?version=" + current_date

Deleting deployment id 3c40e6cf-d29f-4f99-826c-00cf4aa96387
Deleting model id 79fcc6b8-3634-4c19-ab30-bf2cb6500ea3
aidc-spec
Creating package extensions
SUCCESS


#######################################################################################

Synchronous deployment creation for uid: '45308d01-198a-42b5-8a8a-1f30a3072c41' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
......................................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='89d2ea08-e531-4e68-b84c-1df3778b42f1'
------------------------------------------------------------------------------------------------




### Configure Watson OpenScale client

In [7]:
#Configure WOS client
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMetricRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MetricThreshold
from ibm_watson_openscale.supporting_classes.enums import MetricThresholdTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorTagRequest
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Target
from ibm_watson_openscale.supporting_classes.enums import TargetTypes
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import IntegratedSystems, ApplicabilitySelection
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorInstanceSchedule, ScheduleStartTime, MonitorRuntime

from datetime import datetime, timezone, timedelta
import uuid

from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        apikey=WOS_CREDENTIALS['apikey'],
        disable_ssl_verification=True
    )
wos_client = APIClient(service_url=WOS_CREDENTIALS['url'],authenticator=authenticator, service_instance_id=WOS_GUID)

### Configure authentication

In [8]:
auth_type = "bearer" #Supported values are basic and bearer

if auth_type == "basic":
    CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
        "auth_type":"basic",
        "username":  "*****",# update the username here 
        "password": "*****"# Update the password here
   }
    
if auth_type == "bearer":
    CUSTOM_METRICS_PROVIDER_CREDENTIALS = {
        "auth_type":"bearer",
        "token_info": {
            "url": "{}/icp4d-api/v1/authorize".format(WOS_CREDENTIALS['url']),
            "headers": {
                "Content-Type": "application/json",
                "Accept": "application/json"
            },
            "payload": {
                "username": WOS_CREDENTIALS['username'],
                "api_key": WOS_CREDENTIALS['apikey'],
            },
            "method": "post"
        }
    }

### Define the integration system

In [9]:
# Delete existing custom metrics provider integrated systems if present
integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems
for system in integrated_systems:
    if system.entity.type == 'custom_metrics_provider' and system.entity.name == CUSTOM_METRICS_PROVIDER_NAME:
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

#Create new integrated system
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=CUSTOM_METRICS_PROVIDER_NAME,
    description=CUSTOM_METRICS_PROVIDER_NAME,
    type="custom_metrics_provider",
    credentials= CUSTOM_METRICS_PROVIDER_CREDENTIALS,
    connection={
        "display_name": CUSTOM_METRICS_PROVIDER_NAME,
        "endpoint": scoring_url
    }
).result

integrated_system_id = custom_metrics_integrated_system.metadata.id     

def get_custom_monitor_definition():
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    for definition in monitor_definitions:
        if CUSTOM_MONITOR_NAME == definition.entity.name:
            return definition
    return None  

Deleting integrated system AIDC Metrics Provider


### Configure the metrics

In [10]:
# Update the input data type of your model.  
input_data_type = ["structured"]
algorithm_types = ["binary"]
problemTypeSelection = ApplicabilitySelection(problem_type=algorithm_types)
inputDataTypeSelection = ApplicabilitySelection(input_data_type=input_data_type)
monitor_description = 'Set of AIDC Metrics'

#Update the tag values if you want to fetch the metrics by tags
TAGS= ['provider']
TAG_DESCRIPTION =['AIDC']
CUSTOM_MONITOR_METRICS_NAMES = ["decisioncost",'performance', 'impact', 'ml', 'human','roi']
CUSTOM_DESCRIPTION = ['decision cost for the model', 'Average performance of the model', 'Average impact for the model', 'ml percentage', 'human percentage','Return on investment']

### Define thresholds

In [11]:
#Update the Threshold types and default values of the metrics
def custom_metric_definitions():
        
    metrics = [
        MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[0], applies_to=problemTypeSelection,
                             description=CUSTOM_DESCRIPTION[0],
                             thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=-1),
                                       MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=1)]),
        MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[1], applies_to=problemTypeSelection,
                             description=CUSTOM_DESCRIPTION[1],
                             thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=10),
                                         MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=1000)]),
        MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[2], applies_to=problemTypeSelection,
                             description=CUSTOM_DESCRIPTION[2],                               
                             thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=10),
                                         MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=1000)]),
        MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[3], applies_to=problemTypeSelection,
                             description=CUSTOM_DESCRIPTION[3],                                                              
                             thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=40),
                                         MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=80)]),
        MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[4], applies_to=problemTypeSelection,
                             description=CUSTOM_DESCRIPTION[4],
                             thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=20),
                                         MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=50)]),
        MonitorMetricRequest(name=CUSTOM_MONITOR_METRICS_NAMES[5], applies_to=problemTypeSelection,
                             description=CUSTOM_DESCRIPTION[5],
                             thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=100),
                                         MetricThreshold(type=MetricThresholdTypes.UPPER_LIMIT, default=1000.0)])
    ]
    #Comment the below tags code if there are no tags to be created
    tags = [MonitorTagRequest(name=TAGS[0], description=TAG_DESCRIPTION[0])]

    return metrics, tags

### Configure schedule

In [12]:
#Enable the schedule for your custom monitor. Update the value to false if you want to disable the schedule
ENABLE_SCHEDULE = True

#Update the repeat interval and repeat type: Default is 1 hour
repeat_interval = 1
repeat_type = "hour"

#Update the delay unit and interval to trigger the first schedule run after the custom monitor instance is created. Default is 30 minutes
delay_unit = "minute"
delay_time= 30

start_time= ScheduleStartTime(type = "relative", delay_unit= delay_unit, delay = delay_time)
schedule = MonitorInstanceSchedule(repeat_interval=repeat_interval,repeat_unit=repeat_type, start_time=start_time)  

#Update the wait time here.
custom_metrics_wait_time = 50 #time in seconds <update the time here>

#Maximum number of records to consider during custom monitor run evaluation. Update max_records value as per the requirement
max_records = None
#Minimum number of records to consider during custom monitor run evaluation. Update min_records value as per the requirement
min_records = None

### Define the monitor 

In [13]:
def create_custom_monitor_definition():
    # check if the custom monitor definition already exists or not
    existing_definition = get_custom_monitor_definition()

    # if it does not exists, then create a new one.
    if existing_definition is not None:
        print('Deletting the existing monitor definition..')
        wos_client.monitor_definitions.delete(monitor_definition_id = existing_definition.metadata.id, background_mode = False, force = True)
        
    # make sure it is indeed deleted..
    existing_definition = get_custom_monitor_definition()
    if existing_definition is None:
        print('Creating the existing monitor definition..')
        metrics, tags = custom_metric_definitions()
        if ENABLE_SCHEDULE:
            monitor_runtime = MonitorRuntime(type="custom_metrics_provider")
            custom_monitor_details = wos_client.monitor_definitions.add(name=CUSTOM_MONITOR_NAME, description=monitor_description, metrics=metrics, tags=tags, schedule = schedule, 
                                                                        applies_to=inputDataTypeSelection, monitor_runtime = monitor_runtime, background_mode=False).result
        else:
            custom_monitor_details = wos_client.monitor_definitions.add(name=CUSTOM_MONITOR_NAME, description=monitor_description, metrics=metrics, tags=tags, applies_to=inputDataTypeSelection, background_mode=False).result
    else:
        # otherwise, send the existing definition
        print('Monitor deletion did not happen.')
    return custom_monitor_details

custom_monitor_details = create_custom_monitor_definition()
custom_monitor_id = custom_monitor_details.metadata.id

Deletting the existing monitor definition..



 Waiting for end of deleting monitor definition aidc_demo_monitor 




finished

---------------------------------------------------
 Successfully finished deleting monitor definition 
---------------------------------------------------


Creating the existing monitor definition..



 Waiting for end of adding monitor definition aidc_demo_monitor 




finished

-------------------------------------------------
 Successfully finished adding monitor definition 
-------------------------------------------------




### Create the monitor

In [14]:
def get_custom_monitor_instance(custom_monitor_id):
    monitor_instances = wos_client.monitor_instances.list(data_mart_id = WOS_GUID, monitor_definition_id = custom_monitor_id, target_target_id = subscription_id).result.monitor_instances
    if len(monitor_instances) == 1:
        return monitor_instances[0]
    return None

def update_custom_monitor_instance(custom_monitor_instance_id):
    payload = [
     {
       "op": "replace",
       "path": "/parameters",
       "value": {
           "custom_metrics_provider_id": integrated_system_id,
           "custom_metrics_wait_time":   custom_metrics_wait_time,
            "enable_custom_metric_runs": True
       }
     }
    ]
    if max_records is not None:
        payload[0]["value"]["max_records"] = max_records    
    if min_records is not None:
        payload[0]["value"]["min_records"] = min_records 
        
    response = wos_client.monitor_instances.update(custom_monitor_instance_id, payload, update_metadata_only = True)
    result = response.result
    return result

def create_custom_monitor_instance(custom_monitor_id):
    # Check if an custom monitor instance already exists
    existing_monitor_instance = get_custom_monitor_instance(custom_monitor_id)

    # If it does not exist, then create one
    if existing_monitor_instance is None:
        target = Target(
                target_type=TargetTypes.SUBSCRIPTION,
                target_id=subscription_id
            )
        parameters = {
            "custom_metrics_provider_id": integrated_system_id,
            "custom_metrics_wait_time":   custom_metrics_wait_time,
            "enable_custom_metric_runs": True
        }
        # create the custom monitor instance id here.
        custom_monitor_instance_details = wos_client.monitor_instances.create(
                    data_mart_id=WOS_GUID,
                    background_mode=False,
                    monitor_definition_id=custom_monitor_id,
                    target=target,
                    parameters=parameters
        ).result
    else:
        # otherwise, update the existing one with latest integrated system details.
        instance_id = existing_monitor_instance.metadata.id
        custom_monitor_instance_details = update_custom_monitor_instance(instance_id)
    return custom_monitor_instance_details

monitor_instance_details = create_custom_monitor_instance(custom_monitor_id)
custom_monitor_instance_id = monitor_instance_details.metadata.id

def update_monitor_definition_id_in_integrated_systems(custom_monitor_id):
    payload = [
     {
       "op": "add",
       "path": "/parameters",
       "value": {
           "monitor_definition_ids": [ custom_monitor_id ]
     }
     }
    ]
    response = wos_client.integrated_systems.update(integrated_system_id, payload)
    result = response.result
    return result

response = update_monitor_definition_id_in_integrated_systems(custom_monitor_id)




 Waiting for end of monitor instance creation cd09f5db-409b-4244-bcae-25a5123c55b5 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Execute the monitor

In [18]:
#Execute the custom metrics provider deployment
monitor_instance_run_info = wos_client.monitor_instances.run(
        background_mode=False,
        monitor_instance_id=custom_monitor_instance_id
     ).result

monitor_instance_run_info
custom_monitor_run_id = monitor_instance_run_info.metadata.id




 Waiting for end of monitoring run 975dfd43-2f6d-4d12-8440-f253fbcd716b 




running......
finished

---------------------------
 Successfully finished run 
---------------------------




### Display the results

In [19]:
#Display the results
wos_client.monitor_instances.show_metrics(monitor_instance_id=custom_monitor_instance_id)

2023-08-25 15:40:13.212362+00:00,human,4b719e76-a726-4a90-b896-3250643b5c63,17.13,20.0,50.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,975dfd43-2f6d-4d12-8440-f253fbcd716b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:40:13.212362+00:00,roi,4b719e76-a726-4a90-b896-3250643b5c63,433.19521674406684,100.0,1000.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,975dfd43-2f6d-4d12-8440-f253fbcd716b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:40:13.212362+00:00,impact,4b719e76-a726-4a90-b896-3250643b5c63,383.0,10.0,1000.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,975dfd43-2f6d-4d12-8440-f253fbcd716b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:40:13.212362+00:00,ml,4b719e76-a726-4a90-b896-3250643b5c63,82.87,40.0,80.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,975dfd43-2f6d-4d12-8440-f253fbcd716b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:40:13.212362+00:00,decisioncost,4b719e76-a726-4a90-b896-3250643b5c63,-0.04486,-1.0,1.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,975dfd43-2f6d-4d12-8440-f253fbcd716b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:40:13.212362+00:00,performance,4b719e76-a726-4a90-b896-3250643b5c63,382.9,10.0,1000.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,975dfd43-2f6d-4d12-8440-f253fbcd716b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:31:38.653505+00:00,human,9a9071a0-c13d-4f44-97c7-a2ccd8463a53,0.0,20.0,50.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,6199a8ed-47c4-4383-b831-9b6364ee5f5b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:31:38.653505+00:00,roi,9a9071a0-c13d-4f44-97c7-a2ccd8463a53,0.0,100.0,1000.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,6199a8ed-47c4-4383-b831-9b6364ee5f5b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:31:38.653505+00:00,impact,9a9071a0-c13d-4f44-97c7-a2ccd8463a53,0.0,10.0,1000.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,6199a8ed-47c4-4383-b831-9b6364ee5f5b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8
2023-08-25 15:31:38.653505+00:00,ml,9a9071a0-c13d-4f44-97c7-a2ccd8463a53,0.0,40.0,80.0,[],aidc_demo_monitor,cd09f5db-409b-4244-bcae-25a5123c55b5,6199a8ed-47c4-4383-b831-9b6364ee5f5b,subscription,40c1fef9-db6e-4dfb-9eb8-5b4b275b0cb8


Note: First 10 records were displayed.
